In [1]:
import bs4
import requests
import pandas as pd

In [2]:

def scrape_listings(url):
    rows = []
    r = requests.get(url)

    soup = bs4.BeautifulSoup(r.text, 'html.parser')

    listings = soup.find_all('div', {'class': 'result-item'})
    for element in listings:
        row = {}
        row['url'] = element.find('a').get('href')

        url = 'https://www.exchangeandmart.co.uk' + row['url']
        det_url = requests.get(url)
        soup = bs4.BeautifulSoup(det_url.text, 'html.parser')
        name = soup.find('span', {'class': 'ttl'})
        model = soup.find('span', {'class': 'ttlSup'})
        price = soup.find('span', {'itemprop': 'price'})
        row['name'] = str(name.text).strip()
        row['model'] = str(model.text).strip()
        row['price'] = str(price.text).strip()

        facts = element.find('div', {'class': 'key-details'})
        sayi = 1
        for fact in facts.find_all('p'):
            if(sayi != 4 and sayi != 5):
                row[sayi] = str(fact.text).strip()
            sayi+=1
            #add model and name to rows
        
        det1 = soup.find('div', {'class': 'adDetails'})
        det2 = soup.find('div', {'class': 'adDetsColumn'})

        for cell in det1.find_all('div', {'class': 'adDetsItem'}):
            row[sayi] = str(cell.text).strip()
            sayi+=1
        counter = 1
        
        for span in det2.find_all('span'):
            if counter%2 == 0:
                row[sayi] = str(span.text).strip()
                sayi+=1
            counter        

        rows.append(row)

    return pd.DataFrame(rows)


In [3]:
# Solution
df = scrape_listings('https://www.exchangeandmart.co.uk/used-cars-for-sale/bmw/any-distance-from-e1-6an/bodytype-convertible')
df

,url,name,model,price,1,2,3,6,7,8,9,10,11,12,13,14
0,/ad/30929552,BMW 4 Series,2.0 430I M SPORT 2d 242 BHP Convertible,"29,950",2021,Automatic,38.2 mpg,Petrol,2021,2L,"20,955 miles",Petrol,Automatic,Black,Convertible,38.2 mpg
1,/ad/30890858,BMW 2 Series,218i M Sport 2dr [Nav] Convertible,"16,349",2018,Other,47.1 mpg,Petrol,2018,1499cc,"17,440 miles",Petrol,N/A,Black,Convertible,47.1 mpg
2,/ad/30857178,BMW 2 Series,218i M Sport 2dr [Nav] Step Auto Convertible,"17,049",2018,Other,48.7 mpg,Petrol,2018,1499cc,"24,353 miles",Petrol,N/A,Black,Convertible,48.7 mpg
3,/ad/30827473,BMW 4 Series,440i M Sport 2dr Auto [Professional Media] Con...,"26,299",2019,Other,30.4 mpg,Petrol,2019,2998cc,"24,393 miles",Petrol,N/A,White,Convertible,30.4 mpg
4,/ad/30903015,BMW 2 Series,218i Sport 2dr [Nav] Step Auto Convertible,"14,999",2018,Other,50.4 mpg,Petrol,2018,1499cc,"47,476 miles",Petrol,N/A,Blue,Convertible,50.4 mpg
5,/ad/30389317,BMW 2 Series,218i M Sport 2dr [Nav] Step Auto Convertible,"15,699",2019,Other,48.7 mpg,Petrol,2019,1499cc,"38,721 miles",Petrol,N/A,Grey,Convertible,48.7 mpg
6,/ad/29976235,BMW 2 Series,220i Sport 2dr [Nav] Step Auto Convertible,"15,500",2017,Semi-Auto,46.3 mpg,Petrol,2017,2L,"19,389 miles",Petrol,Semiauto,Silver,Convertible,46.3 mpg
7,/ad/30897668,BMW 2 Series,1.5 218i GPF M Sport Convertible 2dr Petrol Au...,"17,700",2019,Automatic,48.7 mpg,Petrol,2019,1499cc,"19,460 miles",Petrol,Automatic,Grey,Convertible,48.7 mpg
8,/ad/30427805,BMW 2 Series,2.0 220i GPF M Sport Convertible 2dr Petrol Au...,"20,000",2020,Automatic,46.3 mpg,Petrol,2020,1998cc,"24,024 miles",Petrol,Automatic,Blue,Convertible,46.3 mpg
9,/ad/30809547,BMW 4 Series,2.0 420d M Sport Auto Euro 6 (s/s) 2dr SAT/NAV,"19,980",2019,Automatic,60.1 mpg,Diesel,2019,1995cc,"20,000 miles",Diesel,Automatic,White,Convertible,60.1 mpg


In [4]:
def scrape_all(pages):
    url = 'https://www.exchangeandmart.co.uk/used-cars-for-sale/bmw/any-distance-from-e1-6an/bodytype-hatchback/page'
    df = pd.DataFrame()
    for page in pages:
        print(page)
        df = pd.concat([df, scrape_listings(url + str(page))], ignore_index=True)
    return df

In [5]:
df_large = scrape_all(range(1, 3))
df_large

1
2


,url,name,model,price,1,2,3,6,7,8,9,10,11,12,13,14
0,/ad/30945065,BMW 1 Series,1.5 118I SE 5d 139 BHP Hatchback,"16,550",2019,Automatic,43.5 mpg,Petrol,2019,1.5L,"21,000 miles",Petrol,Automatic,Black,Hatchback,43.5 mpg
1,/ad/30618095,BMW 1 Series,118i [1.5] M Sport Shadow Ed 5dr Step Auto Hat...,"19,490",2019,Semi-Auto,54.3 mpg,Petrol,2019,1.5L,"22,000 miles",Petrol,Semiauto,Grey,Hatchback,54.3 mpg
2,/ad/30852995,BMW 1 Series,118i M Sport 5dr [Tech Pack] Hatchback,"22,649",2020,Other,41.5 mpg,Petrol,2020,1499cc,"10,535 miles",Petrol,N/A,Black,Hatchback,41.5 mpg
3,/ad/30755082,BMW 1 Series,116d Sport 5dr Hatchback,"19,599",2020,Other,57.7 mpg,Diesel,2020,1496cc,"11,102 miles",Diesel,N/A,White,Hatchback,57.7 mpg
4,/ad/30953353,BMW i3,125kW 42kWh 5dr Auto [Loft Interior World] Hat...,"18,899",2020,Other,Tax: £0,2020,"12,340 miles",Electric,N/A,Grey,Hatchback,NaN,NaN,NaN
5,/ad/30931119,BMW 4 Series,420i M Sport 5dr Step Auto Hatchback,"34,949",2022,Other,39.2 mpg,Petrol,2022,1998cc,"13,840 miles",Petrol,N/A,White,Hatchback,39.2 mpg
6,/ad/30572345,BMW 1 Series,118i M Sport 5dr Step Auto Hatchback,"21,349",2020,Other,40.4 mpg,Petrol,2020,1499cc,"13,896 miles",Petrol,N/A,Blue,Hatchback,40.4 mpg
7,/ad/30498579,BMW 1 Series,118i M Sport 5dr Step Auto Hatchback,"21,449",2020,Other,40.4 mpg,Petrol,2020,1499cc,"14,265 miles",Petrol,N/A,Blue,Hatchback,40.4 mpg
8,/ad/30890914,BMW 1 Series,118i M Sport 5dr Hatchback,"20,699",2020,Other,41.5 mpg,Petrol,2020,1499cc,"14,594 miles",Petrol,N/A,White,Hatchback,41.5 mpg
9,/ad/30679036,BMW 1 Series,118i M Sport 5dr Step Auto Hatchback,"21,999",2020,Other,40.4 mpg,Petrol,2020,1499cc,"15,276 miles",Petrol,N/A,Blue,Hatchback,40.4 mpg


In [6]:
df_large.to_csv('bmw convertible.csv', index=False)